The adversarial attack begins

In order to do any kind of deanonimizaiton, we would want to match the public registry data lines with the anonimized data lines. We know the names of all the people in anonymized dataset (the survey records were leaked). We want to match those names to learn the politcal preference of these people.

Let's first replicate other groups variable transformation (or how I like to call it categorization), so that our data format is the same, and we can actually try matching the data on same properties. 

In [24]:
import pandas as pd
from datetime import datetime

PATH = r"data/new/filtered_data_registerZ.xlsx"
NEW = r"data/new/variable_adjusted_filtered_data_registerZ.xlsx"

# Load the Excel sheet into a DataFrame
df = pd.read_excel(PATH, sheet_name='Sheet1')

# Track the number of changes for each operation
#citizenship_changes = (df['citizenship'] != "Denmark").sum()

# Replace non-Denmark citizenships with "Foreign"
df['citizenship'] = df['citizenship'].apply(lambda x: "Foreign" if x != "Denmark" else x)

# Log the number of rows modified for citizenship
#print(f"{citizenship_changes} rows were modified for citizenship")


# Ensure the 'dob' column is a datetime object (if not, convert it)
if not pd.api.types.is_datetime64_any_dtype(df['dob']):
    df['dob'] = pd.to_datetime(df['dob'], errors='coerce')

# Transform dob column to age
df['dob'] = pd.to_datetime(df['dob'])
df['age'] = df['dob'].apply(lambda x: datetime.now().year - x.year)

# Ensure the 'dob' column is formatted as date only (no time part)
df['dob'] = df['dob'].dt.date

# Define the bins and labels for age groups
bins = [20, 40, 60, 100]
labels = ['20-40', '40-60', '60+']

# Create a new column 'age_group' based on the defined bins
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

# Display the updated DataFrame
print(df[['name', 'age', 'age_group']])


                   name  age age_group
0            Lee, Allen   21     20-40
1       Bhagria, Pranav   22     20-40
2        Mills, Brandon   21     20-40
3            Crow, Chad   22     20-40
4           Lopez, Jose   22     20-40
..                  ...  ...       ...
195  Verhoeven, Santana   86       60+
196         Ware, Aaron   73       60+
197     Harris, Dchelle   64       60+
198          Sar, Dylan   70       60+
199        Yellow, Adam   51     40-60

[200 rows x 3 columns]


In [25]:
df['marital_status'] = df['marital_status'].apply(
    lambda x: "Not married" if x in ["Divorced", "Widowed", "Never married"] else "Married/separate"
    )

df.drop(columns=['last_voted', 'age', 'dob', 'zip'], inplace=True)

In [26]:
# Save to xlsx file hehe 😈

df.to_excel(NEW, index=False)